In [3]:
import zipfile
with zipfile.ZipFile("yelp_academic_dataset_user.json.zip", "r") as z:
    z.extractall("yelp")


In [25]:
with zipfile.ZipFile("yelp_academic_dataset_review.json.zip", "r") as z:
    z.extractall("yelp")

raw_data = sc.textFile("hdfs://ec2-52-43-197-246.us-west-2.compute.amazonaws.com:9000/yelp/yelp/yelp_academic_dataset_review.json")
#raw_data = spark.read.load("hdfs://ec2-52-43-197-246.us-west-2.compute.amazonaws.com:9000/yelp/yelp/yelp_academic_dataset_review.json", format="json")

In [70]:
from pyspark import SparkConf, SparkContext, SparkFiles, SQLContext


In [72]:
sqlContext = SQLContext(sc)
path = "hdfs://ec2-52-43-197-246.us-west-2.compute.amazonaws.com:9000/yelp/yelp/yelp_academic_dataset_review.json"

In [75]:
raw_data = sqlContext.read.json(path)

In [79]:
raw_data.printSchema()


root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- useful: long (nullable = true)



In [112]:
data = raw_data.select("user_id", "business_id", "stars")

In [117]:
data.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)



In [118]:
training_RDD, validation_RDD, test_RDD = data.randomSplit([0.6, 0.2, 0.2], seed=0L)
training_RDD.cache()
validation_RDD.cache()
test_RDD.cache()

DataFrame[user_id: string, business_id: string, stars: bigint]

In [119]:
from pyspark.mllib.recommendation import ALS
import math
seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02
min_error = float('inf')
best_rank = -1
best_iteration = -1

In [128]:
training_RDD.show()

+--------------------+--------------------+-----+
|             user_id|         business_id|stars|
+--------------------+--------------------+-----+
|--4TkB_iDShmg41Y_...|JImJictLIE48_O1z-...|    4|
|--4TkB_iDShmg41Y_...|PkG9bV1SvfOydOCCn...|    5|
|--4TkB_iDShmg41Y_...|qWYdBWci87q7JRZen...|    1|
|--65q1FpAL_UQtVZ2...|0vzZ_Bcb02rJljeMU...|    3|
|--65q1FpAL_UQtVZ2...|14NSxaXBNspIVnsrM...|    2|
|--65q1FpAL_UQtVZ2...|18TUn9oiW0k0yB6lh...|    3|
|--65q1FpAL_UQtVZ2...|3UMcsN9IjoeV8JZvC...|    4|
|--65q1FpAL_UQtVZ2...|3UMcsN9IjoeV8JZvC...|    4|
|--65q1FpAL_UQtVZ2...|AkOruz5CrCxUmXe1p...|    4|
|--65q1FpAL_UQtVZ2...|AkOruz5CrCxUmXe1p...|    4|
|--65q1FpAL_UQtVZ2...|Cp6JGY5YIRncTV_My...|    4|
|--65q1FpAL_UQtVZ2...|CqhsIZ7TzCfJTrZqg...|    5|
|--65q1FpAL_UQtVZ2...|DeAnVirgOxQnI36V1...|    3|
|--65q1FpAL_UQtVZ2...|L_MtTn4IUBTmQtqqM...|    4|
|--65q1FpAL_UQtVZ2...|Pfb6VOIiroqDWOebf...|    4|
|--65q1FpAL_UQtVZ2...|PxeX1M8WtyPxX1Muu...|    4|
|--65q1FpAL_UQtVZ2...|apCfJVnRyaw6oVUBK...|    5|


In [129]:
model = ALS.train(training_RDD, 2, 1)

Py4JJavaError: An error occurred while calling o1016.trainALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 73.0 failed 4 times, most recent failure: Lost task 6.3 in stage 73.0 (TID 550, ip-172-31-0-101.us-west-2.compute.internal): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 267, in dump_stream
    bytes = self.serializer.dumps(vs)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 415, in dumps
    return pickle.dumps(obj, protocol)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/mllib/recommendation.py", line 44, in __reduce__
    return Ratin

In [94]:
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

Py4JJavaError: An error occurred while calling o484.trainALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 23.0 failed 4 times, most recent failure: Lost task 3.3 in stage 23.0 (TID 94, ip-172-31-0-102.us-west-2.compute.internal): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 267, in dump_stream
    bytes = self.serializer.dumps(vs)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 415, in dumps
    return pickle.dumps(obj, protocol)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/mllib/recommendation.py", line 44, in __reduce__
    return Rating,

In [122]:
sc.version


u'1.6.1'